This is a starter notebook for the [Kitchenware classification](https://www.kaggle.com/competitions/kitchenware-classification) competition on Kaggle

To get started:

- Join the competition and accept rules
- Download your Kaggle credentials file
- If you're running in Saturn Cloud, configure your instance to have access to access the kaggle credentials

When this is done, we can download the data. We need to execute the following cell only once

In [1]:
!kaggle competitions download -c kitchenware-classification
!mkdir data
!unzip kitchenware-classification.zip -d data > /dev/null
!rm kitchenware-classification.zip

'kaggle' is not recognized as an internal or external command,
operable program or batch file.
A subdirectory or file data already exists.
The system cannot find the path specified.
'rm' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


Now let's train a baseline model

In [3]:
import mlflow

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras

In [4]:
DATASET_SIZE = 9367
IMAGE_SIZE = 224
BATCH_SIZE = 8
WORKERS = 4
EPOCHS = 10

BASE_PATH='./data'

classes = [
    'cup', 
    'fork', 
    'glass', 
    'knife', 
    'plate', 
    'spoon'
]

First, we will load the training dataframe and split it into train and validation

In [5]:
df_train_full = pd.read_csv('data/train.csv', dtype={'Id': str})
df_train_full['filename'] = 'data/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

,Id,label,filename
0,0560,glass,data/images/0560.jpg
1,4675,cup,data/images/4675.jpg
2,0875,glass,data/images/0875.jpg
3,4436,spoon,data/images/4436.jpg
4,8265,plate,data/images/8265.jpg


In [6]:
val_cutoff = int(len(df_train_full) * 0.8)
df_train = df_train_full[:val_cutoff]
df_val = df_train_full[val_cutoff:]

Now let's create image generators

In [7]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

Found 4447 validated image filenames belonging to 6 classes.
Found 1112 validated image filenames belonging to 6 classes.


In [9]:
classes = np.array(list(train_generator.class_indices.keys()))
classes

array(['cup', 'fork', 'glass', 'knife', 'plate', 'spoon'], dtype='<U5')

In [10]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 min_delta = 1e-4,
                                                 patience = 5,
                                                 mode = 'max',
                                                 restore_best_weights = True,
                                                 verbose = 1)

callbacks = [earlystopping]

In [11]:
EXP_NAME = 'model_search_transformers_large'
mlflow.set_experiment(EXP_NAME)
mlflow.set_tracking_uri("sqlite:///mlruns.db")
mlflow.tensorflow.autolog()

2022/12/15 13:28:42 INFO mlflow.tracking.fluent: Experiment with name 'model_search_transformers_large' does not exist. Creating a new experiment.
2022/12/15 13:28:42 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


In [12]:
from keras_cv_attention_models import beit, efficientnet, coatnet, davit, convnext, hornet, swin_transformer_v2, maxvit

In [13]:
base_model = maxvit.MaxViT_Tiny(    # MaxViT_Small(
    pretrained='imagenet',  #weights='imagenet',
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    num_classes=len(classes)
)
base_model.trainable = False
base_model = tf.keras.Model(inputs=base_model.layers[1].input, outputs= base_model.layers[-4].output)

inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
model = tf.keras.Sequential([
    inputs,
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(12, 'gelu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6, 'softmax')
],
name = 'MaxViT_Base')

c:\Users\andre\anaconda3\envs\keras-attention-models\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


>>>> Load pretrained from: C:\Users\andre\.keras\models\maxvit_tiny_224_imagenet.h5


In [14]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [15]:
history = model.fit(
    x = train_generator,
    validation_data=val_generator,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    workers=WORKERS,
    callbacks=callbacks
)

2022/12/15 13:28:50 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/12/15 13:28:51 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

Epoch 1/10
556/556 [==============================] - 91s 127ms/step - loss: 0.3281 - accuracy: 0.8972 - val_loss: 0.1366 - val_accuracy: 0.9541
Epoch 2/10
556/556 [==============================] - 67s 121ms/step - loss: 0.2080 - accuracy: 0.9357 - val_loss: 0.1927 - val_accuracy: 0.9469
Epoch 3/10
556/556 [==============================] - 67s 121ms/step - loss: 0.1812 - accuracy: 0.9406 - val_loss: 0.1381 - val_accuracy: 0.9577
Epoch 4/10
556/556 [==============================] - 67s 121ms/step - loss: 0.1666 - accuracy: 0.9458 - val_loss: 0.1533 - val_accuracy: 0.9622
Epoch 5/10
556/556 [==============================] - 68s 122ms/step - loss: 0.1694 - accuracy: 0.9424 - val_loss: 0.1387 - val_accuracy: 0.9649
Epoch 6/10
556/556 [==============================] - 67s 121ms/step - loss: 0.1481 - accuracy: 0.9494 - val_loss: 0.2404 - val_accuracy: 0.9550
Epoch 7/10
556/556 [==============================] - 68s 121ms/step - loss: 0.1565 - accuracy: 0.9530 - val_loss: 0.1439 - val_ac